# Линейные модели. Задачи регрессии и классификации.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, SGDClassifier, Ridge, Lasso

## Линейная регрессия

### Подготовка данных

In [ ]:
df = pd.read_excel('Concrete_Data.xls', sheet_name='Sheet1')

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
df = df.rename(lambda x: x.split('(')[0].strip().replace(' ', '_').lower(), axis=1)

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(df.corr(), annot=True, cmap='icefire');

In [ ]:
sns.pairplot(df, y_vars='concrete_compressive_strength');

In [ ]:
df.head()

### Подготовка выборок

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['concrete_compressive_strength'], axis=1),
                                                    df['concrete_compressive_strength'],
                                                    test_size=0.2,
                                                    random_state=177013,
                                                    shuffle=True,
                                                    )

In [ ]:
X_train_b = X_train.copy()
X_train_b['bias'] = 1

X_test_b = X_test.copy()
X_test_b['bias'] = 1

In [ ]:
weights = np.linalg.inv(X_train_b.T @ X_train_b) @ X_train_b.T @ y_train

In [ ]:
weights.rename(lambda x: df.columns[x] if x != len(weights) - 1 else 'bias')

In [ ]:
y_pred = X_test_b @ weights.values

In [ ]:
y_pred

## Метрики регрессии

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### Среднеквадратичная ошибка (MSE)

In [ ]:
((y_test - y_pred)**2).mean()

In [ ]:
((y_test - y_train.mean())**2).mean()

In [ ]:
mean_squared_error(y_test, y_pred)

### Средняя абсолютная ошибка (MAE)

In [ ]:
(abs(y_test - y_pred)).mean()

In [ ]:
mean_absolute_error(y_test, y_pred)

### Коэффициент детерминации ($R^2$)

In [ ]:
1 - mean_squared_error(y_test, y_pred) / y_test.var(ddof=0)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
def show_regression_metrics(y_true, y_pred):
    print(f'MSE: {mean_squared_error(y_true, y_pred)}')
    print(f'MAE: {mean_absolute_error(y_true, y_pred)}')
    print(f'R2: {r2_score(y_true, y_pred)}')

## Линейная регрессия в sklearn

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
show_regression_metrics(y_test, y_pred)

## Градиентный спуск

In [ ]:
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train_b = scaler.transform(X_train)
X_train_b = np.hstack([X_train_b, np.ones((len(X_train), 1))])

In [ ]:
X_test_b = scaler.transform(X_test)
X_test_b = np.hstack([X_test_b, np.ones((len(X_test), 1))])

In [ ]:
weights = np.linalg.inv(X_train_b.T @ X_train_b) @ X_train_b.T @ y_train

In [ ]:
pd.Series(weights, index=[df.columns[x] for x in range(len(weights) - 1)] + ['bias'])

$$
\frac{\partial MSE}{\partial w} = \frac{\partial MSE}{\partial y_{pred}} \frac{\partial y_{pred}}{\partial w}
$$

$$
\frac{\partial MSE}{\partial y_{pred}} = \frac{\partial(y-y_{pred})^2}{\partial y_{pred}} = \frac{\partial(y^2-2y\cdot y_{pred}+y_{pred}^2)}{\partial y_{pred}} = 2(y_{pred} - y)
$$

$$
\frac{\partial y_{pred}}{\partial w} = \frac{(\partial X \cdot w)}{\partial w} = X
$$

In [ ]:
num_iterations = 100
learning_rate = 1e-4
msetrain = []
msetest = []
checkpoints = []

improved_weights = np.zeros(len(X_train.columns) + 1)

for _ in range(num_iterations):
    preds = X_train_b @ improved_weights
    
    # d(MSE) / d(preds)
    error = preds - y_train 
    msetrain.append(mean_squared_error(y_train, preds))
    msetest.append(mean_squared_error(y_test, X_test_b @ improved_weights))
    
    # d(MSE) / d(weights)
    gradient = X_train_b.T @ error
    
    improved_weights -= learning_rate * gradient
    checkpoints.append(improved_weights)

In [ ]:
weights

In [ ]:
improved_weights

In [ ]:
plt.plot(msetrain);
plt.plot(msetest);
plt.title('Функция потерь')
plt.xlabel('Итерация')
plt.ylabel('MSE');
plt.legend(['Обучающая выборка', 'Валидационная выборка']);

In [ ]:
y_pred = X_test_b @ improved_weights

In [ ]:
show_regression_metrics(y_test, y_pred)

In [ ]:
np.argmin(msetest)

In [ ]:
final_weights = checkpoints[np.argmin(msetest)]

In [ ]:
y_pred = X_test_b @ final_weights

In [ ]:
show_regression_metrics(y_test, y_pred)

### Градиентный спуск в sklearn

In [ ]:
sgd = SGDRegressor(random_state=177013, penalty=None)

In [ ]:
sgd.fit(X_train_b, y_train)

In [ ]:
y_pred = sgd.predict(X_test_b)

In [ ]:
show_regression_metrics(y_test, y_pred)

## Допущения линейной регрессии

- Линейность: подразумевается наличие линейной зависимости (но вы можете преобразовать признаки, чтобы ее обеспечить!).
- Слабая экзогенность: выборка не должна зывисеть от внешних факторов, погрешность измерений скажется на погрешности предсказаний.
- Отсутствие мультиколлинеарности: признаки не должны сильно коррелировать.
- Нормальность ошибки: ошибка предсказания должна распределяться нормально (и иметь матожидание около нуля):

In [ ]:
plt.hist(y_train - lr.predict(X_train), bins='fd');
plt.title('Распределение ошибок')
plt.xlabel('Ошибка предсказания')
plt.ylabel('Количество, шт');

- Гомоскедастичность: дисперсия ошибки должна быть постоянной:

In [ ]:
plt.scatter(y_train, (y_train - lr.predict(X_train)) ** 2);
plt.axhline(np.mean((y_train - lr.predict(X_train)) ** 2), color='red', linestyle='--')
plt.title('Распределение ошибок')
plt.xlabel('Истинное значение')
plt.ylabel('Ошибка$^2$');

- Независимость ошибок: проверяется с помощью DW-теста (значения статистики в норме от 1.5 до 2.5):

In [ ]:
from statsmodels.stats.stattools import durbin_watson
durbin_watson(y_train - lr.predict(X_train))

Большинство проблем можно решить одним махом, преобразовав входные данные с целью уменьшить их асимметрию. Обычное шкалирование почти не влияет на аналитическое решение, но необходимо для градиентного спуска. Однако есть методы, которые не только нормализуют данные, но и корректируют их форму:

- `PowerTransformer` - параметрический преобразователь, стабилизирующий дисперсию и уменьшающий скошенность методом максимального правдоподобия;
- `QuantileTransformer` - непараметрический преобразователь, превращающий распределение в равномерное или нормальное через PPF.

Платой за такое удобство является некоторое искажение линейных зависимостей.

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
for scaler in [
                StandardScaler(),
                QuantileTransformer(output_distribution='uniform', random_state=177013),
                QuantileTransformer(output_distribution='normal', random_state=177013),
                PowerTransformer(),
              ]:

    pipe = Pipeline([
                        ('scaler', scaler),
                        ('model', LinearRegression()),
                    ])
    
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(f'Предобработка: {scaler}')
    show_regression_metrics(y_test, y_pred)
    print()

## Регуляризация

In [ ]:
for model in [
                Lasso(random_state=177013),
                Ridge(random_state=177013),
              ]:

    pipe = Pipeline([
                        ('scaler', PowerTransformer()),
                        ('model', model),
                    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(f'Модель: {model}, веса: {model.coef_}')
    show_regression_metrics(y_test, y_pred)
    print()

# Логистическая регрессия

## Подготовка данных

In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

In [ ]:
df['Sex'] = (df['Sex'] == 'male').astype('int')

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap='icefire');

In [ ]:
sns.pairplot(df[['Age', 'Fare']]);

In [ ]:
df.groupby('Pclass')['Age'].agg(['count', 'mean', 'median'])

In [ ]:
df.groupby('SibSp')['Age'].agg(['count', 'mean', 'median', lambda x: x.isna().sum()])

In [ ]:
df['Age'] = df['Age'].fillna(df.groupby(['Pclass', 'SibSp'])['Age'].transform('median'))
df = df.dropna()

In [ ]:
df.head()

## Разбиение на выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Survived', axis=1),
                                                    df['Survived'],
                                                    test_size=0.2,
                                                    random_state=177013,
                                                    shuffle=True,
                                                    stratify=df['Survived'])

In [ ]:
X_train_b = X_train.copy()
X_train_b['bias'] = 1

X_test_b = X_test.copy()
X_test_b['bias'] = 1

In [ ]:
weights = np.linalg.inv(X_train_b.T @ X_train_b) @ X_train_b.T @ y_train

In [ ]:
weights

In [ ]:
y_logits = X_test_b @ weights.values

In [ ]:
y_logits.describe()

In [ ]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [ ]:
y_probs = y_logits.apply(sigmoid)

In [ ]:
((y_probs > 0.5) == y_test).mean()

In [ ]:
1 - df['Survived'].mean()

In [ ]:
((y_probs > 0.62) == y_test).mean()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, (y_probs > 0.62))

## Метрики классификации

In [ ]:
from sklearn.metrics import (f1_score, roc_auc_score, average_precision_score, confusion_matrix, roc_curve,
                             precision_recall_curve, classification_report, recall_score, precision_score,
                             log_loss, brier_score_loss)

### Метрики для оценки моделей

Для оценки статистической верности моделей нужна метрика, которая будет:

- классонезависимой;
- порогонезависимой.

В качестве альтернативного варианта можно уравнять число представителей класса в обучающей выборке (oversampling, undersampling, параметр `class_weights` в sklearn), чтобы иметь возможность использовать интуитивные метрики с порогом 0.5. Такой подход имеет массу недостатков:

 - четкий баланс классов еще не гарантирует, что 0.5 будет оптимальным порогом;
 - на каждый эксперимент с пропорцией требуется переобучить модель;
 - модель более склонна "уезжать" в продакшене.
 
Поэтому **если от вас прямо не требуют обратного, оценивайте модели по "честным" метрикам, а "интуитивные" оставьте для презентации заказчику.**

#### Log loss (кросс-энтропия)

In [ ]:
-(y_test * np.log(y_probs) + (1 - y_test) * np.log(1-y_probs)).mean()

In [ ]:
log_loss(y_test, y_probs)

#### Показатель Брайера для бинарной классификации

In [ ]:
mean_squared_error(y_test, y_probs)

In [ ]:
brier_score_loss(y_test, y_probs)

### Метрики для презентации. Матрица ошибок.

In [ ]:
def calculate_metrics(target_test, probabilities):
    cmatrix = confusion_matrix(target_test, probabilities > 0.5)

    ap = average_precision_score(target_test, probabilities)
    fpr, tpr, _ = roc_curve(target_test, probabilities)
    roc_auc = roc_auc_score(target_test, probabilities)

    precision, recall, thresholds = precision_recall_curve(target_test, probabilities)
    f_scores = 2 * recall * precision / (recall + precision)
    best_thresh = thresholds[np.argmax(f_scores)]
    best_f = np.max(f_scores)
    best_acc = accuracy_score(target_test, (probabilities > best_thresh))
    best_cmatrix = confusion_matrix(target_test, (probabilities > best_thresh))

    return best_f, roc_auc, best_acc, ap, best_thresh, fpr, tpr, recall, precision, cmatrix, best_cmatrix

In [ ]:
def visualize(target_test, probabilities):
    fig, axes = plt.subplots(1, 2, figsize=(12,5))
    axes[0].plot([0, 1], linestyle='--')
    axes[1].plot([0.5, 0.5], linestyle='--')

    best_f, roc_auc, acc, ap, best_thresh, fpr, tpr, recall, precision, cmatrix, best_cmatrix = calculate_metrics(target_test, probabilities)
    print(f'ROC_AUC: {roc_auc:.2f}, AP (PR_AUC): {ap:.2f}, наилучший F1: {best_f:.2f} с порогом {best_thresh:.2f} (accuracy {acc:.2f})')
    axes[0].plot (fpr, tpr);
    axes[1].plot (recall, precision);

    axes[0].set (xlabel='FPR', ylabel='TPR', title='ROC-кривая', xlim=(0,1), ylim=(0,1))
    axes[1].set (xlabel='Recall', ylabel='Precision', title='PR-кривая', xlim=(0,1), ylim=(0,1))
    plt.show()
    fig, axes = plt.subplots(1, 2, figsize=(12,4))
    sns.heatmap(cmatrix, ax=axes[0], annot=True, cmap='Blues', fmt='d').set(title='Матрица ошибок', xlabel='Предсказание', ylabel='Реальность')
    sns.heatmap(best_cmatrix, ax=axes[1], annot=True, cmap='Blues', fmt='d').set(title='Матрица ошибок (оптимальный порог)', xlabel='Предсказание', ylabel='Реальность')

    
    return best_thresh

#### Порогозависимые метрики.

На каждом возможном пороге (а их число равно числу уникальных скоров в `predict_proba()` + 1 (как правило, это порого, равный нулю, на котором все классифицируется как положительный класс)) существует своя матрица ошибок и как следствие, много производных от нее метрик:

- **accuracy**: она нам уже знакома, это просто процент верно угаданных случаев;
- **recall**, он же **TPR** (True Positive Rate): TP/(TP + FN) - процент от реальных случаев положительного класса, которые модель обнаружила;
- **precision**: TP / (TP + FP) - процент от предсказанных случаев положительного класса, которые были верными;
- **F1**: среднегармоническое recall и precision.

#### Порогонезависимые метрики. ROC-кривая. PR-кривая.

Метрики, суммарные по всем порогам положительного класса, будут порогонезависимы (хотя и классоспецифичны). Наиболее часто используются площади под параметрическими кривыми:

- **ROC_AUC**: строится кривая FRP (FP/TN + FP) и TPR по всем порогам. Считается площадь под кривой.
- **Average precision (PR_AUC)**: строится кривая recall и precision по всем порогам. Считается площадь под кривой.

Также мы можем с их помощью выбрать порог, где соотношение этих метрик оптимально (например, дает наибольшую F1).

In [ ]:
visualize(y_test, y_probs);

**Подбор оптимального порога относится к этапу принятия решения. Его следует проводить в финале исследования (ведь на практике может быть, к примеру, и несколько порогов с разными действиями).**

## Логистическая регрессия в sklearn

In [ ]:
lr = LogisticRegression(random_state=177013, penalty=None)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_probs = lr.predict_proba(X_test)

In [ ]:
y_probs[:5]

In [ ]:
visualize(y_test, y_probs[:,1]);

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

## Градиентный спуск

In [ ]:
scaler.fit(X_train)
X_train_b = scaler.transform(X_train)
X_train_b = np.hstack([X_train_b, np.ones((len(X_train), 1))])
X_test_b = scaler.transform(X_test)
X_test_b = np.hstack([X_test_b, np.ones((len(X_test), 1))])

In [ ]:
num_iterations = 100
learning_rate = 1e-4
nlltrain = []
nlltest = []
checkpoints = []

improved_weights = np.zeros(len(X_train.columns) + 1)

for _ in range(num_iterations):
    logits = X_train_b @ improved_weights
    
    probabilities = sigmoid(logits)
    
    error = probabilities - y_train
    nlltrain.append(log_loss(y_train, probabilities).mean())
    nlltest.append(log_loss(y_test, sigmoid(X_test_b @ improved_weights)).mean())
    
    gradient = X_train_b.T @ error
    
    improved_weights -= learning_rate * gradient
    checkpoints.append(improved_weights)

In [ ]:
improved_weights

In [ ]:
plt.plot(nlltrain);
plt.plot(nlltest);
plt.title('Функция потерь')
plt.xlabel('Итерация')
plt.ylabel('NLL');
plt.legend(['Обучающая выборка', 'Валидационная выборка']);

In [ ]:
y_logits = X_test_b @ improved_weights
y_probs = sigmoid(y_logits)

In [ ]:
accuracy_score(y_test, (y_probs > 0.5))

In [ ]:
np.argmin(msetest)

In [ ]:
final_weights = checkpoints[np.argmin(msetest)]

In [ ]:
y_logits = X_test_b @ final_weights
y_probs = sigmoid(y_logits)

In [ ]:
visualize(y_test, y_probs);

### Градиентный спуск в sklearn

In [ ]:
sgd = SGDClassifier(random_state=177013, penalty=None, loss='log_loss', learning_rate='constant', eta0=1e-3)

In [ ]:
sgd.fit(X_train_b, y_train)

In [ ]:
y_probs = sgd.predict_proba(X_test_b)
y_pred = sgd.predict(X_test_b)

In [ ]:
visualize(y_test, y_probs[:,1]);

## Регуляризация

In [ ]:
lr = LogisticRegression(random_state=177013, penalty='l2')

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_probs = lr.predict_proba(X_test)

In [ ]:
y_probs[:5]

In [ ]:
visualize(y_test, y_probs[:,1]);

## Допущения логистической регрессии

По сравнению с обычной линейной регрессией, логистическая НЕ требует:

- гомоскедастичности;
- нормальности ошибок;
- поскольку результат проходит через логистическую функцию, линейная зависимость нужна для логитов, а не самого целевого признака, который бинарен.

# Домашнее задание

## Easy

С помощью sklearn можно генерировать простые датасеты для тестирования моделей:

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=20, n_informative=10, n_classes=2)
X_train, y_train = X[:8000], y[:8000]
X_test, y_test = X[8000:], y[8000:]

Воспользуйтесь примером из урока и постройте модель логистической регрессии. Проверьте метрику F1 с порогом по умолчанию.

In [ ]:
# Ваш код ниже:


## Normal

Вернитесь к датасету о клиентах банка. Подумайте, как преобразовать категории в числа (если не справитесь, можете их удалить, мы рассмотрим этот вопрос подробнее на следующем уроке).

Выделите обучающую и валидационную выборки.

Постройте модель логистической регрессии для предсказания признака `Exited`.

In [ ]:
# Ваш код ниже:


Проверьте метрику F1 с порогом по умолчанию. Попробуйте найти хороший порог классификации с помощью PR-кривой.

In [ ]:
# Ваш код ниже:


Напишите краткий вывод. Хорошо ли работает модель? Если нет, то как вы думаете, почему?

Вывод: 

## Hard

In [ ]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=10000, n_features=20, n_informative=10)
X_train, y_train = X[:8000], y[:8000]
X_test, y_test = X[8000:], y[8000:]

1. Постройте модель линейной регрессии. Оцените метрику на тестовой выборке.

In [ ]:
# Ваш код ниже:


2. Сгенерируйте случайную матрицу 20х20 (можно воспользоваться `sklearn.datasets.make_spd_matrix()`). Проверьте ее на невырожденность (`np.linalg.det()` не должен равняться 0).

    Умножьте X_train и X_test на эту матрицу. Постройте модель линейной регрессии по этим преобразованным данным.

    Проверьте метрику. Что вы наблюдаете?

In [ ]:
# Ваш код ниже:


С помощью линейной алгебры и формул линейной регрессии объясните, почему так происходит.